# High-level Keras (TF) Example

In [1]:
# Parameters
EPOCHS=5
N_CLASSES=10
BATCHSIZE=64
LR=0.01
MOMENTUM=0.9
GPU=True

In [2]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from utils import cifar_for_library, yield_mb
from nb_logging import NotebookLogger, output_to, error_to
from os import path

Using TensorFlow backend.


In [3]:
nb_teminal_logger = NotebookLogger(sys.stdout.session, sys.stdout.pub_thread, sys.stdout.name, sys.__stdout__)

In [4]:
rst_out = output_to(nb_teminal_logger)
rst_err = error_to(nb_teminal_logger)

In [5]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
# Channels should be first (otherwise slow)
print(K.backend.image_data_format())

OS:  linux
Python:  3.5.4 |Anaconda, Inc.| (default, Oct  5 2017, 08:00:22) 
[GCC 7.2.0]
Keras:  2.0.8
Numpy:  1.13.3
Tensorflow:  1.3.0
tensorflow
channels_last


In [6]:
data_path = path.join(os.getenv('AZ_LEARNING_INPUT_DATASET'), 'cifar-10-batches-py')

In [7]:
def create_symbol():
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(N_CLASSES, activation='softmax'))
    return model

In [8]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

In [9]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(data_path, channel_first=False, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...


Preparing test set...
Done.


(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 812 ms, sys: 736 ms, total: 1.55 s
Wall time: 1.54 s


In [10]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 96 ms, sys: 8 ms, total: 104 ms
Wall time: 106 ms


In [11]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 23.3 ms


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 50)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 50)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 50)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 50)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 100)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 100)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 100)         0         
__________

In [13]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/5


   64/50000 [..............................] - ETA: 2301s - loss: 2.2981 - acc: 0.0781

  128/50000 [..............................] - ETA: 1184s - loss: 2.3069 - acc: 0.0859

  256/50000 [..............................] - ETA: 613s - loss: 2.3101 - acc: 0.0820 

  384/50000 [..............................] - ETA: 417s - loss: 2.3110 - acc: 0.0833

...

49600/50000 [============================>.] - ETA: 0s - loss: 0.8724 - acc: 0.6925

49792/50000 [============================>.] - ETA: 0s - loss: 0.8720 - acc: 0.6927

49984/50000 [============================>.] - ETA: 0s - loss: 0.8719 - acc: 0.6927

50000/50000 [==============================] - 21s - loss: 0.8719 - acc: 0.6927    


CPU times: user 52.7 s, sys: 33 s, total: 1min 25s
Wall time: 1min 53s


In [14]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 796 ms, sys: 572 ms, total: 1.37 s
Wall time: 1.7 s


In [15]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.7284
